In [1]:
# Run this cell for every experiment
import pandas as pd
import numpy as np
import pickle
import platform
from sklearn.preprocessing import StandardScaler

from mabwiser.mab import MAB
from mabwiser.linear import _RidgeRegression, _Linear

class LinTSExample(_RidgeRegression):
    def predict(self, x):
        if self.scaler is not None:
            x = self._scale_predict_context(x) 
        beta_sampled = rng.multivariate_normal(x, self.A_inv)
        return np.dot(x, beta_sampled)
    
class LinearExample(_Linear):
    factory = {"ts": LinTSExample}

    def __init__(self, rng, arms, n_jobs=1, backend=None, l2_lambda=1, alpha=1, regression='ts', arm_to_scaler = None):
        super().__init__(rng, arms, n_jobs, backend, l2_lambda, alpha, regression)
       
        self.l2_lambda = l2_lambda
        self.alpha = alpha
        self.regression = regression

        # Create ridge regression model for each arm
        self.num_features = None

        if arm_to_scaler is None:
            arm_to_scaler = dict((arm, None) for arm in arms)

        self.arm_to_model = dict((arm, LinearExample.factory.get(regression)(rng, l2_lambda,
                                                                       alpha, arm_to_scaler[arm])) for arm in arms)


# Create Data Set

In [2]:
from sklearn.datasets import make_multilabel_classification

X, y = make_multilabel_classification(n_samples=1000, n_features=25, n_classes=20, n_labels=3, length=50, 
                                      allow_unlabeled=True, sparse=False, return_indicator='dense', 
                                      return_distributions=False, random_state=11)

In [3]:
y[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])

In [4]:
data = pd.DataFrame(X, columns=['feature_'+str(i) for i in range(25)])
responses = pd.DataFrame(y, columns=['item_'+str(i) for i in range(20)])

In [5]:
data = data.merge(responses, how='left', left_index=True, right_index=True)
data.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,item_10,item_11,item_12,item_13,item_14,item_15,item_16,item_17,item_18,item_19
0,0.0,1.0,1.0,2.0,3.0,3.0,2.0,1.0,4.0,6.0,...,0,0,0,0,0,1,0,0,0,0
1,3.0,2.0,1.0,4.0,1.0,2.0,3.0,1.0,3.0,2.0,...,0,0,0,0,0,1,0,0,0,1
2,2.0,4.0,3.0,4.0,3.0,5.0,3.0,2.0,1.0,3.0,...,1,0,0,0,0,1,0,0,0,0
3,0.0,3.0,4.0,0.0,1.0,1.0,1.0,1.0,1.0,6.0,...,0,0,1,0,0,1,0,1,0,0
4,3.0,1.0,1.0,1.0,3.0,3.0,0.0,1.0,1.0,1.0,...,1,0,0,0,0,0,0,0,0,0


In [6]:
data['user'] = [i for i in range(1000)]

In [7]:
data.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,item_11,item_12,item_13,item_14,item_15,item_16,item_17,item_18,item_19,user
0,0.0,1.0,1.0,2.0,3.0,3.0,2.0,1.0,4.0,6.0,...,0,0,0,0,1,0,0,0,0,0
1,3.0,2.0,1.0,4.0,1.0,2.0,3.0,1.0,3.0,2.0,...,0,0,0,0,1,0,0,0,1,1
2,2.0,4.0,3.0,4.0,3.0,5.0,3.0,2.0,1.0,3.0,...,0,0,0,0,1,0,0,0,0,2
3,0.0,3.0,4.0,0.0,1.0,1.0,1.0,1.0,1.0,6.0,...,0,1,0,0,1,0,1,0,0,3
4,3.0,1.0,1.0,1.0,3.0,3.0,0.0,1.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,4


In [8]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, random_state=43, test_size=0.3)

In [9]:
id_vars = ['user'] + ['feature_'+str(i) for i in range(25)]
train = train.melt(id_vars=id_vars, var_name='item_id', value_name='response')
train.head()

,user,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,item_id,response
0,651,3.0,2.0,3.0,2.0,2.0,2.0,5.0,0.0,2.0,...,2.0,1.0,0.0,6.0,2.0,4.0,2.0,1.0,item_0,0
1,789,3.0,0.0,2.0,3.0,5.0,2.0,1.0,1.0,1.0,...,0.0,5.0,5.0,3.0,2.0,2.0,0.0,1.0,item_0,0
2,609,1.0,5.0,4.0,3.0,3.0,4.0,0.0,1.0,4.0,...,2.0,2.0,2.0,3.0,3.0,4.0,3.0,0.0,item_0,0
3,506,2.0,8.0,5.0,2.0,3.0,1.0,1.0,1.0,2.0,...,0.0,3.0,2.0,4.0,1.0,3.0,2.0,1.0,item_0,0
4,847,4.0,0.0,2.0,7.0,2.0,2.0,1.0,1.0,4.0,...,1.0,3.0,0.0,0.0,5.0,3.0,1.0,0.0,item_0,0


In [10]:
test = test.melt(id_vars=id_vars, var_name='item_id', value_name='response')
test.head()

,user,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,item_id,response
0,858,3.0,1.0,2.0,2.0,3.0,3.0,2.0,3.0,2.0,...,3.0,4.0,1.0,1.0,2.0,2.0,1.0,2.0,item_0,0
1,986,0.0,1.0,2.0,0.0,1.0,1.0,2.0,0.0,1.0,...,3.0,2.0,2.0,2.0,5.0,2.0,1.0,2.0,item_0,1
2,183,0.0,2.0,0.0,2.0,0.0,2.0,1.0,2.0,6.0,...,5.0,0.0,3.0,4.0,7.0,1.0,1.0,0.0,item_0,0
3,502,0.0,0.0,1.0,0.0,2.0,1.0,2.0,1.0,1.0,...,0.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,item_0,1
4,710,0.0,3.0,0.0,0.0,0.0,3.0,3.0,3.0,1.0,...,4.0,2.0,1.0,4.0,0.0,1.0,1.0,2.0,item_0,0


In [11]:
# Check that there are train positives
train.groupby('item_id')['response'].sum()

item_id
item_0      40
item_1       8
item_10    191
item_11     27
item_12    209
item_13    211
item_14     43
item_15    135
item_16      5
item_17     26
item_18     84
item_19     47
item_2     131
item_3     177
item_4     111
item_5     129
item_6       7
item_7     124
item_8     210
item_9     200
Name: response, dtype: int64

In [12]:
test.groupby('item_id')['response'].sum()

item_id
item_0     28
item_1      2
item_10    78
item_11    16
item_12    95
item_13    80
item_14    24
item_15    67
item_16     2
item_17     8
item_18    32
item_19    20
item_2     54
item_3     69
item_4     45
item_5     47
item_6      1
item_7     46
item_8     91
item_9     80
Name: response, dtype: int64

In [13]:
context_features = ['feature_'+str(i) for i in range(25)]

decisions = MAB._convert_array(train['item_id'])
rewards = MAB._convert_array(train['response'])
contexts = MAB._convert_matrix(train[context_features]).astype('float')

test_contexts = MAB._convert_matrix(test[context_features]).astype('float')

In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
contexts = scaler.fit_transform(contexts)

In [15]:
rng = np.random.RandomState(seed=11)
arms = [i for i in range(20)]
mab = LinearExample(rng=rng, arms=arms, l2_lambda=1, alpha=1, regression='ts', n_jobs=1, backend=None)

mab.fit(decisions, rewards, contexts)

for arm in mab.arms:
    u, s, vh = np.linalg.svd(mab.arm_to_model[arm].A_inv)
    print(s)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 

The data has duplicate singular values, and will thus be able to reproduce the non-deterministic behavior

In [17]:
context_features

['feature_0',
 'feature_1',
 'feature_2',
 'feature_3',
 'feature_4',
 'feature_5',
 'feature_6',
 'feature_7',
 'feature_8',
 'feature_9',
 'feature_10',
 'feature_11',
 'feature_12',
 'feature_13',
 'feature_14',
 'feature_15',
 'feature_16',
 'feature_17',
 'feature_18',
 'feature_19',
 'feature_20',
 'feature_21',
 'feature_22',
 'feature_23',
 'feature_24']

In [18]:
train['set'] = 'train'
test['set'] = 'test'

data = pd.concat([train, test])
data.to_csv('simulated_multi_label_data.csv', index=False)

# Mac OS

In [2]:
platform.platform()

'macOS-10.15.7-x86_64-i386-64bit'

In [3]:
print(np.__version__)

1.18.5


In [4]:
data = pd.read_csv('simulated_multi_label_data.csv')

train = data[data['set']=='train']
test = data[data['set']=='test']

context_features = ['feature_'+str(i) for i in range(25)]

decisions = MAB._convert_array(train['item_id'])
rewards = MAB._convert_array(train['response'])
contexts = MAB._convert_matrix(train[context_features]).astype('float')

# Drop duplicate rows from test for prediction
test_unique = test.drop_duplicates(subset=['user']+context_features).reset_index(drop=True)
test_contexts = MAB._convert_matrix(test_unique[context_features]).astype('float')


In [5]:
scaler = StandardScaler()
contexts = scaler.fit_transform(contexts)
test_contexts = scaler.transform(test_contexts)

In [6]:
rng = np.random.RandomState(seed=11)
arms = [i for i in range(20)]

mab = LinearExample(rng=rng, arms=arms, l2_lambda=1, alpha=1, regression='ts', n_jobs=1, backend=None)
mab.arm_to_model[1]

In [7]:
mab.fit(decisions, rewards, contexts)
expectations = mab.predict_expectations(test_contexts)

expectations[0][1]

8.609566768600269

In [8]:
expectations = pd.DataFrame(expectations)
test_unique = test_unique.merge(expectations, how='left', left_index=True, right_index=True)
test_unique.drop(context_features+['item_id', 'response', 'set'], inplace=True, axis=1)

In [9]:
test_unique.head()

,user,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,858,6.179651,8.609567,7.552786,5.865039,7.628488,6.191944,15.690249,9.941365,12.944682,...,11.008013,12.931333,5.907228,7.199660,6.240309,3.567583,13.689466,5.754822,6.345826,5.889495
1,986,32.269648,41.113554,38.195166,35.655877,33.274402,21.576775,45.871261,40.705381,29.660084,...,28.600415,28.654465,36.719987,34.597348,36.461886,28.009544,41.879546,35.280315,35.575348,30.654710
2,183,45.883117,47.157955,37.983692,40.815035,44.395965,49.322856,44.973318,41.633995,43.031945,...,30.900773,34.582317,34.026266,34.664748,37.962249,50.869745,48.970175,34.419763,34.778202,34.896416
3,502,12.011752,14.958983,18.138486,4.557334,9.289095,17.869559,13.324098,10.439434,13.301328,...,8.659405,21.440960,17.061643,15.036238,17.755798,13.904548,15.789957,12.813377,9.595818,14.731508
4,710,36.303692,38.663870,44.740352,39.306088,21.562238,45.569861,33.847269,23.456933,30.433636,...,37.293819,31.718610,28.369931,33.783749,29.221677,35.022899,39.332997,39.703151,32.459510,43.705576


In [10]:
pickle.dump(mab, open('mac_multi_mab.pkl', 'wb'))
test_unique.to_csv('mac_multi_expectations.csv', index=False)
pickle.dump(scaler, open('multi_scaler.pkl', 'wb'))

# Windows

In [ ]:
platform.platform()

In [ ]:
print(np.__version__)

In [ ]:
data = pd.read_csv('simulated_multi_label_data.csv')

train = data[data['set']=='train']
test = data[data['set']=='test']

context_features = ['feature_'+str(i) for i in range(25)]

decisions = MAB._convert_array(train['item_id'])
rewards = MAB._convert_array(train['response'])
contexts = MAB._convert_matrix(train[context_features]).astype('float')

# Drop duplicate rows from test for prediction
test_unique = test.drop_duplicates(subset=['user']+context_features).reset_index(drop=True)
test_contexts = MAB._convert_matrix(test_unique[context_features]).astype('float')


In [ ]:
scaler = pickle.load(open('multi_scaler.pkl', 'rb'))

contexts = scaler.transform(contexts)
test_contexts = scaler.transform(test_contexts)

In [ ]:
rng = np.random.RandomState(seed=11)
arms = [i for i in range(20)]

mab = LinearExample(rng=rng, arms=arms, l2_lambda=1, alpha=1, regression='ts', n_jobs=1, backend=None)
mab.arm_to_model[1]

In [ ]:
mab.fit(decisions, rewards, contexts)
expectations = mab.predict_expectations(test_contexts)

expectations[0][1]

In [ ]:
expectations = pd.DataFrame(expectations)
test_unique = test_unique.merge(expectations, how='left', left_index=True, right_index=True)
test_unique.drop(context_features+['item_id', 'response', 'set'], inplace=True, axis=1)

In [ ]:
pickle.dump(mab, open('win_multi_mab.pkl', 'wb'))
test_unique.to_csv('win_multi_expectations.csv', index=False)
